In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time, os

In [3]:
fbref_query = "https://fbref.com/en/comps/9/history/Premier-League-Seasons"

In [4]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(result.text, "html.parser")
    return soup

In [5]:
getAndParseURL(fbref_query)


<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202311031" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '

### From 1950 to 2023

In [6]:
seasons = []
html = getAndParseURL(fbref_query)
season_count=34
for season in html.findAll("td",{"data-stat":"league_name"}):
    seasons.append("https://fbref.com" + season.find("a").get("href"))
    season_count -= 1
    if season_count == 0:
        break
        
seasons


['https://fbref.com/en/comps/9/Premier-League-Stats',
 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier

In [7]:
seasons

['https://fbref.com/en/comps/9/Premier-League-Stats',
 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier

In [8]:
data = requests.get(seasons[1])
current_df = pd.read_html(data.text, match="Squad Standard Stats")[0]

In [9]:
current_df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Squad               # Pl                Age   
0             Arsenal                 26               24.7   
1         Aston Villa                 26               27.0   
2         Bournemouth                 31               26.3   
3           Brentford                 25               26.2   
4            Brighton                 29               26.3   
5             Chelsea                 32               26.3   
6      Crystal Palace                 26               26.7   
7             Everton                 28               26.6   
8              Fulham                 29               28.2   
9        Leeds United                 29               25.3   
10     Leicester City                 28               26.1   
11          Liverpool                 28               27.4   
12    Manchester City                 24               27.2   
13     Manchester Utd                 26               26.9   
14      Newcastle Utd                 27               27.3   
15    Nott'ham Forest                 33               26.5   
16        Southampton                 36               24.7   
17          Tottenham                 29               27.6   
18           West Ham                 25               28.2   
19             Wolves                 32               26.7   

   Unnamed: 3_level_0 Playing Time                    Performance      ...  \
                 Poss           MP Starts   Min   90s         Gls Ast  ...   
0                59.3           38    418  3420  38.0          84  64  ...   
1                49.3           38    418  3420  38.0          49  35  ...   
2                40.4           38    418  3420  38.0          37  24  ...   
3                43.8           38    418  3420  38.0          56  36  ...   
4                60.2           38    418  3420  38.0          68  46  ...   
5                58.7           38    418  3420  38.0          37  27  ...   
6                46.3           38    418  3420  38.0          38  29  ...   
7                42.8           38    418  3420  38.0          32  24  ...   
8                48.8           38    418  3420  38.0          52  33  ...   
9                47.0           38    418  3420  38.0          45  31  ...   
10               47.7           38    418  3420  38.0          49  36  ...   
11               60.8           38    418  3420  38.0          71  58  ...   
12               64.7           38    418  3420  38.0          92  69  ...   
13               53.7           38    418  3420  38.0          56  42  ...   
14               52.3           38    418  3420  38.0          64  42  ...   
15               37.6           38    418  3420  38.0          36  22  ...   
16               44.5           38    418  3420  38.0          36  25  ...   
17               50.0           38    418  3420  38.0          68  46  ...   
18               42.1           38    418  3420  38.0          41  25  ...   
19               49.9           38    418  3420  38.0          28  12  ...   

   Per 90 Minutes                                                             
              Gls   Ast   G+A  G-PK G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG  
0            2.21  1.68  3.89  2.13   3.82  1.89  1.41   3.31  1.82     3.23  
1            1.29  0.92  2.21  1.21   2.13  1.32  1.02   2.34  1.24     2.26  
2            0.97  0.63  1.61  0.97   1.61  1.02  0.75   1.76  1.02     1.76  
3            1.47  0.95  2.42  1.29   2.24  1.49  1.02   2.51  1.33     2.35  
4            1.79  1.21  3.00  1.63   2.84  1.93  1.37   3.30  1.81     3.19  
5            0.97  0.71  1.68  0.89   1.61  1.30  1.02   2.32  1.24     2.26  
6            1.00  0.76  1.76  0.97   1.74  1.03  0.76   1.80  0.97     1.73  
7            0.84  0.63  1.47  0.76   1.39  1.19  0.85   2.04  1.13     1.98  
8            1.37  0.87  2.24  1.24   2.11  1.22  0.75   1.97  1.03     1.78  
9            1.18  0.82  2.00 

In [10]:
all_dfs = []
for season in seasons:
    try:
        data = requests.get(season)
        current_df = pd.read_html(data.text, match="Squad Standard Stats")[0]
        all_dfs.append(current_df)
    except:
        print(season)
    time.sleep(5)

https://fbref.com/en/comps/9/1991-1992/1991-1992-Premier-League-Stats
https://fbref.com/en/comps/9/1990-1991/1990-1991-Premier-League-Stats
https://fbref.com/en/comps/9/1989-1990/1989-1990-Premier-League-Stats
https://fbref.com/en/comps/9/1988-1989/1988-1989-Premier-League-Stats
https://fbref.com/en/comps/9/1987-1988/1987-1988-Premier-League-Stats
https://fbref.com/en/comps/9/1986-1987/1986-1987-Premier-League-Stats
https://fbref.com/en/comps/9/1985-1986/1985-1986-Premier-League-Stats
https://fbref.com/en/comps/9/1984-1985/1984-1985-Premier-League-Stats
https://fbref.com/en/comps/9/1983-1984/1983-1984-Premier-League-Stats
https://fbref.com/en/comps/9/1982-1983/1982-1983-Premier-League-Stats
https://fbref.com/en/comps/9/1981-1982/1981-1982-Premier-League-Stats
https://fbref.com/en/comps/9/1980-1981/1980-1981-Premier-League-Stats
https://fbref.com/en/comps/9/1979-1980/1979-1980-Premier-League-Stats
https://fbref.com/en/comps/9/1978-1979/1978-1979-Premier-League-Stats
https://fbref.com/en

In [11]:
final_result = pd.concat(all_dfs, axis=0, ignore_index=True)

NameError: name 'all_dfs' is not defined

In [12]:
final_result

Expected                      Per 90 Minutes                           \
        npxG npxG+xAG   xAG    xG            Ast   G+A G+A-PK  G-PK   Gls   
0       16.5     29.3  12.7  21.0           1.17  3.25   2.75  1.58  2.08   
1       21.2     37.9  16.7  23.5           1.50  3.67   3.42  1.92  2.17   
2       13.8     23.1   9.3  13.8           0.50  1.42   1.42  0.92  0.92   
3       20.8     37.5  16.7  22.1           1.08  2.58   2.42  1.33  1.50   
4       20.0     36.5  16.5  20.8           1.50  3.42   3.33  1.83  1.92   
..       ...      ...   ...   ...            ...   ...    ...   ...   ...   
641      NaN      NaN   NaN   NaN           0.86  2.05   2.02  1.17  1.19   
642      NaN      NaN   NaN   NaN           0.95  2.24   2.17  1.21  1.29   
643      NaN      NaN   NaN   NaN           0.90  2.19   2.14  1.24  1.29   
644      NaN      NaN   NaN   NaN           0.79  2.17   2.17  1.38  1.38   
645      NaN      NaN   NaN   NaN           0.95  2.24   2.19  1.24  1.29   

           ... Playing Time                  Progression         \
     npxG  ...          90s  MP   Min Starts        PrgC   PrgP   
0    1.38  ...         12.0  12  1080    132       245.0  630.0   
1    1.77  ...         12.0  12  1080    132       254.0  496.0   
2    1.15  ...         12.0  12  1080    132       197.0  352.0   
3    1.73  ...         12.0  12  1080    132       172.0  456.0   
4    1.67  ...         12.0  12  1080    132       284.0  546.0   
..    ...  ...          ...  ..   ...    ...         ...    ...   
641   NaN  ...         42.0  42  3780    440         NaN    NaN   
642   NaN  ...         42.0  42  3780    463         NaN    NaN   
643   NaN  ...         42.0  42  3780    455         NaN    NaN   
644   NaN  ...         42.0  42  3780    454         NaN    NaN   
645   NaN  ...         42.0  42  3780    452         NaN    NaN   

    Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 Squad               # Pl                Age   
0              Arsenal                 23               25.2   
1          Aston Villa                 23               27.1   
2          Bournemouth                 25               25.7   
3            Brentford                 25               26.6   
4             Brighton                 25               27.2   
..                 ...                ...                ...   
641      Sheffield Utd                 23               25.8   
642     Sheffield Weds                 25               27.8   
643        Southampton                 25               26.3   
644          Tottenham                 31               24.9   
645          Wimbledon                 27               25.9   

    Unnamed: 3_level_0  
                  Poss  
0                 60.7  
1                 53.8  
2                 42.9  
3                 47.7  
4                 61.4  
..                 ...  
641                NaN  
642                NaN  
643                NaN  
644                NaN  
645                NaN  

[646 rows x 32 columns]

In [10]:
final_result.columns = final_result.columns.droplevel()

NameError: name 'final_result' is not defined

In [14]:
final_result

,npxG,npxG+xAG,xAG,xG,Ast,G+A,G+A-PK,G-PK,Gls,npxG,...,90s,MP,Min,Starts,PrgC,PrgP,Squad,# Pl,Age,Poss
0,16.5,29.3,12.7,21.0,1.17,3.25,2.75,1.58,2.08,1.38,...,12.0,12,1080,132,245.0,630.0,Arsenal,23,25.2,60.7
1,21.2,37.9,16.7,23.5,1.50,3.67,3.42,1.92,2.17,1.77,...,12.0,12,1080,132,254.0,496.0,Aston Villa,23,27.1,53.8
2,13.8,23.1,9.3,13.8,0.50,1.42,1.42,0.92,0.92,1.15,...,12.0,12,1080,132,197.0,352.0,Bournemouth,25,25.7,42.9
3,20.8,37.5,16.7,22.1,1.08,2.58,2.42,1.33,1.50,1.73,...,12.0,12,1080,132,172.0,456.0,Brentford,25,26.6,47.7
4,20.0,36.5,16.5,20.8,1.50,3.42,3.33,1.83,1.92,1.67,...,12.0,12,1080,132,284.0,546.0,Brighton,25,27.2,61.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,NaN,NaN,NaN,NaN,0.86,2.05,2.02,1.17,1.19,NaN,...,42.0,42,3780,440,NaN,NaN,Sheffield Utd,23,25.8,NaN
642,NaN,NaN,NaN,NaN,0.95,2.24,2.17,1.21,1.29,NaN,...,42.0,42,3780,463,NaN,NaN,Sheffield Weds,25,27.8,NaN
643,NaN,NaN,NaN,NaN,0.90,2.19,2.14,1.24,1.29,NaN,...,42.0,42,3780,455,NaN,NaN,Southampton,25,26.3,NaN
644,NaN,NaN,NaN,NaN,0.79,2.17,2.17,1.38,1.38,NaN,...,42.0,42,3780,454,NaN,NaN,Tottenham,31,24.9,NaN


In [15]:
final_result.to_csv("SquadStandardStats.csv")

In [ ]:
final_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Expected, npxG)             140 non-null    float64
 1   (Expected, npxG+xAG)         140 non-null    float64
 2   (Expected, xAG)              140 non-null    float64
 3   (Expected, xG)               140 non-null    float64
 4   (Per 90 Minutes, Ast)        580 non-null    float64
 5   (Per 90 Minutes, G+A)        580 non-null    float64
 6   (Per 90 Minutes, G+A-PK)     580 non-null    float64
 7   (Per 90 Minutes, G-PK)       580 non-null    float64
 8   (Per 90 Minutes, Gls)        580 non-null    float64
 9   (Per 90 Minutes, npxG)       140 non-null    float64
 10  (Per 90 Minutes, npxG+xAG)   140 non-null    float64
 11  (Per 90 Minutes, xAG)        140 non-null    float64
 12  (Per 90 Minutes, xG)         140 non-null    float64
 13  (Per 90 Minutes, xG+

In [1]:
def getTables(matchTable,seasons,table):
    for season in seasons:
        try:
            data = requests.get(season)
            current_df = pd.read_html(data.text, match=matchTable)[0]
            table.append(current_df)
        except:
            print(season)
        time.sleep(5)

In [12]:
shooting = []
passing = []
goal_shot_creation = []
defensive_action = []
possession = []
#table_list = ["shooting","passing","goal_shot_creation","defensive_action","possession"]
#matching_list = [""]

In [13]:
getTables("Squad Shooting",seasons,shooting)

https://fbref.com/en/comps/9/1991-1992/1991-1992-Premier-League-Stats
https://fbref.com/en/comps/9/1990-1991/1990-1991-Premier-League-Stats


In [15]:
shooting_results = pd.concat(shooting, axis=0, ignore_index=True)

In [16]:
shooting_results

Expected                             Standard                          \
        G-xG np:G-xG  npxG npxG/Sh    xG     Dist   FK  G/Sh G/SoT Gls PK   
0        4.0     2.5  16.5    0.10  21.0     16.5  0.0  0.11  0.35  25  6   
1        2.5     1.8  21.2    0.13  23.5     15.9  3.0  0.13  0.35  26  3   
2       -2.8    -2.8  13.8    0.10  13.8     17.1  6.0  0.08  0.22  11  0   
3       -4.1    -4.8  20.8    0.12  22.1     14.9  7.0  0.09  0.31  18  2   
4        2.2     2.0  20.0    0.12  20.8     17.1  5.0  0.13  0.31  23  1   
..       ...     ...   ...     ...   ...      ...  ...   ...   ...  .. ..   
641      NaN     NaN   NaN     NaN   NaN      NaN  NaN   NaN  0.23  50  1   
642      NaN     NaN   NaN     NaN   NaN      NaN  NaN   NaN  0.24  54  3   
643      NaN     NaN   NaN     NaN   NaN      NaN  NaN   NaN  0.25  54  2   
644      NaN     NaN   NaN     NaN   NaN      NaN  NaN   NaN  0.39  58  0   
645      NaN     NaN   NaN     NaN   NaN      NaN  NaN   NaN  0.26  54  2   

                                          Unnamed: 0_level_0  \
    PKatt     Sh  Sh/90  SoT  SoT% SoT/90              Squad   
0       6  168.0  14.00   55  32.7   4.58            Arsenal   
1       3  176.0  14.67   66  37.5   5.50        Aston Villa   
2       0  138.0  11.50   50  36.2   4.17        Bournemouth   
3       2  173.0  14.42   51  29.5   4.25          Brentford   
4       1  171.0  14.25   70  40.9   5.83           Brighton   
..    ...    ...    ...  ...   ...    ...                ...   
641     2    NaN    NaN  209   NaN   4.98      Sheffield Utd   
642     3    NaN    NaN  216   NaN   5.14     Sheffield Weds   
643     2    NaN    NaN  205   NaN   4.88        Southampton   
644     0    NaN    NaN  149   NaN   3.55          Tottenham   
645     3    NaN    NaN  198   NaN   4.71          Wimbledon   

    Unnamed: 1_level_0 Unnamed: 2_level_0  
                  # Pl                90s  
0                   23               12.0  
1                   23               12.0  
2                   25               12.0  
3                   25               12.0  
4                   25               12.0  
..                 ...                ...  
641                 23               42.0  
642                 25               42.0  
643                 25               42.0  
644                 31               42.0  
645                 27               42.0  

[646 rows x 20 columns]

In [17]:
shooting_results.to_csv("shooting.csv")

In [18]:
getTables("Squad Passing",seasons,passing)
passing_results = pd.concat(passing, axis=0, ignore_index=True)
passing_results.to_csv("passing.csv")

https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats
https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats
https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats
https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats
https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats
https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats
https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier-League-Stats
https://fbref.com/en/comps/9/2009-2010/2009-2010-Premier-League-Stats
https://fbref.com/en/comps/9/2008-2009/2008-2009-Premier-League-Stats
https://fbref.com/en/comps/9/2007-2008/2007-2008-Premier-League-Stats
https://fbref.com/en/comps/9/2006-2007/2006-2007-Premier-League-Stats
https://fbref.com/en/comps/9/2005-2006/2005-2006-Premier-League-Stats
https://fbref.com/en/comps/9/2004-2005/2004-2005-Premier-League-Stats
https://fbref.com/en/comps/9/2003-2004/2003-2004-Premier-League-Stats
https://fbref.com/en

In [21]:
getTables("Squad Possession",seasons,possession)
possession_results = pd.concat(possession, axis=0, ignore_index=True)
possession_results.to_csv("possession.csv")

https://fbref.com/en/comps/9/2016-2017/2016-2017-Premier-League-Stats
https://fbref.com/en/comps/9/2015-2016/2015-2016-Premier-League-Stats
https://fbref.com/en/comps/9/2014-2015/2014-2015-Premier-League-Stats
https://fbref.com/en/comps/9/2013-2014/2013-2014-Premier-League-Stats
https://fbref.com/en/comps/9/2012-2013/2012-2013-Premier-League-Stats
https://fbref.com/en/comps/9/2011-2012/2011-2012-Premier-League-Stats
https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier-League-Stats
https://fbref.com/en/comps/9/2009-2010/2009-2010-Premier-League-Stats
https://fbref.com/en/comps/9/2008-2009/2008-2009-Premier-League-Stats
https://fbref.com/en/comps/9/2007-2008/2007-2008-Premier-League-Stats
https://fbref.com/en/comps/9/2006-2007/2006-2007-Premier-League-Stats
https://fbref.com/en/comps/9/2005-2006/2005-2006-Premier-League-Stats
https://fbref.com/en/comps/9/2004-2005/2004-2005-Premier-League-Stats
https://fbref.com/en/comps/9/2003-2004/2003-2004-Premier-League-Stats
https://fbref.com/en

In [ ]:
# otomatik hale getir
#def get_write_data(tables,seasons,matchList): 

In [28]:
#len(all_dfs_standings)

74

In [29]:
#final_result_standings = pd.concat(all_dfs_standings, axis=0, ignore_index=True)

In [30]:
final_result_standings

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Manchester City,12,9,1,2,32,12,20,28,2.33,24.4,11.1,13.3,1.11,L W W W D,42627.0,Erling Haaland - 13,Ederson,NaN
1,2,Liverpool,12,8,3,1,27,10,17,27,2.25,26.5,14.5,12.0,1.00,D W W D W,50648.0,Mohamed Salah - 10,Alisson,NaN
2,3,Arsenal,12,8,3,1,26,10,16,27,2.25,21.0,9.2,11.8,0.99,W D W L W,60130.0,Eddie Nketiah - 5,David Raya,NaN
3,4,Tottenham,12,8,2,2,24,15,9,26,2.17,19.2,18.6,0.6,0.05,W W W L L,61726.0,Son Heung-min - 8,Guglielmo Vicario,NaN
4,5,Aston Villa,12,8,1,3,29,17,12,25,2.08,23.5,15.9,7.7,0.64,D W W L W,41378.0,Ollie Watkins - 6,Emiliano Martínez,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,18,Fulham,42,13,11,18,52,68,-16,37,0.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1559,19,Huddersfield,42,15,6,21,64,92,-28,36,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1560,20,Chelsea,42,12,8,22,53,65,-12,32,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1561,21,Sheffield Weds,42,12,8,22,64,83,-19,32,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Relegated


In [31]:
final_result_standings.to_csv("RegularSeason.csv")

In [ ]:
all_dfs_standings = []
for season in seasons:
    try:
        data = requests.get(season)
        current_df = pd.read_html(data.text, match="Regular season")[0]
        all_dfs_standings.append(current_df)
    except:
        print(season)
    time.sleep(5)

### --------------------

In [5]:
soup = getAndParseURL('https://fbref.com/en/comps/9/Premier-League-Stats')

In [11]:
soup


<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202311031" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '

In [18]:
soup = getAndParseURL('https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats')

In [19]:
soup.find("div",{"class":"comps"}).h1.text.strip().split()[0]

'2022-2023'